# Hikurangi Ruptures Demo (using ipyvolume)

Use the **ipyvolume** library to produce an interactive 3D visualisation of Hikurangi w Ruptures. 

**Goals**

 - we can share these visualisations easily, and they should 'just work' on any O/S + common web browser
 - team members can experiment and customise these demos with no risk and a small learning curve.
 - demos can run on the free **mybinder.org** service, simply by sharing the binder link.
 
**Known issues** 

 - doesn't work with the **Microsoft Edge** browser

In [2]:
import ipyvolume as ipv
import geopandas as gpd
import requests
import numpy as np

In [3]:
#for data animations
import asyncio
import nest_asyncio
nest_asyncio.apply()

In [4]:
# tile outlines shapefiles  - choose your resolution
URL30 = "https://github.com/GNS-Science/eq-fault-geom/blob/master/data/subduction/tile_outlines_30.zip?raw=true"
URL10 = "https://github.com/GNS-Science/eq-fault-geom/blob/master/data/subduction/tile_outlines.zip?raw=true"
tile_outlines_nztm = gpd.read_file(URL10).to_crs(epsg=2193)

## Build the tri-polygon coords and indices need for ipyvolume

We're using the **ipyvolume.plot_trisurf()** function which builds surfaces from triangles (tsurfs). Each tile in the subductio zone is a surface comprising two equilateral triangles on a single plane. 

 - build three lists for the x,y,z coordinates of the corner locations
 - calculate the indices for the x,y,z co-ordinates of each tile and store this in the dataframe

In [5]:
def quadPolyToTriPoly(coords):
    """
    return the x,y & z lists defining define the tile corners in 3d space.
    """
    assert len(coords) == 5
    x = [g[0] for g in coords[:-1]]
    y = [g[1] for g in coords[:-1]]
    z = [g[2] for g in coords[:-1]]
    return x, y, z

# build the complete fault section tile coordinate lists 
x,y,z = [], [], []
for bdry in tile_outlines_nztm.geometry.boundary:
    x0,y0,z0 = quadPolyToTriPoly(list(bdry.coords))
    x.extend(x0)
    y.extend(y0)
    z.extend(z0)

In [6]:
# add the triangle indices to the dataframe
def calculateFaceIdices(geometry):
    #using this trick https://stackoverflow.com/a/18317089 to get the index
    idx = int(geometry.name) *4
    faces = [[idx, idx+1, idx+2], [idx, idx+2, idx+3]]
    return faces
    
tile_outlines_nztm['triangles'] = tile_outlines_nztm['geometry'].to_frame().apply(calculateFaceIdices, axis=1)

## Build the rectangular ruptures

Use the **fault_section** module to create some rupture sets to superimpose on our tiled surface.

In [7]:
# import our python code 
from io import BytesIO
import codecs
import requests
module_uri = "https://raw.githubusercontent.com/GNS-Science/eq-fault-geom/master/src/eq_fault_geom/geomops/rupture_set/fault_section.py"
url = requests.get(module_uri, allow_redirects=True)
bytesio_object = BytesIO(url.content)
# Write the stuff
with open("fault_section.py", "wb") as f:
    f.write(bytesio_object.getbuffer())

import fault_section
from fault_section import FaultSubSectionFactory, SheetFault

In [8]:
# Configure the rupture builder to use the tile sources as used for the surfaces

URL30 = "https://github.com/GNS-Science/eq-fault-geom/blob/master/data/subduction/tile_parameters_30.csv?raw=true"
URL10 = "https://github.com/GNS-Science/eq-fault-geom/blob/master/data/subduction/tile_parameters.csv?raw=true"
tile_params = BytesIO(requests.get(URL10).content)
StreamReader = codecs.getreader('utf-8')
wrapper_file = StreamReader(tile_params)

factory = FaultSubSectionFactory()
sf = SheetFault("Hikurangi")\
        .build_surface_from_csv(factory, wrapper_file)

In [9]:
# Variations on 'nearly rectangular' rupture sets ..
#
# scale: determines ratio of rectangle to original tile size (10km*10km).
# aspect: ratio of 'along-strike' tiles to 'down-dip' tiles per rectangle.
# interhttps://hub.gke.mybinder.org/user/chrisbc-379a8fb-0a027b8c1b4eff1-75eu0sje/notebooks/Hikurangi%20Rupture%20Demo.ipynb#val: how many tiles to advance both col-wise and row-wise
# min_fill_factor: how many tiles are needed to make up a valid 'rectangle'
#
spec0 = dict(scale=3, aspect=2)
spec1 = dict(scale=3, aspect=4, interval=2, min_fill_factor=0.5) #default min_fill_factor is 0.75
spec2 = dict(scale=8, aspect=2, interval=4, min_fill_factor=0.55)
spec3 = dict(scale=16, aspect=1, interval=4, min_fill_factor=0.33)
spec4 = dict(scale=16, aspect=2, interval=4, min_fill_factor=0.33)
spec5 = dict(scale=3, aspect=12, interval=2, min_fill_factor=0.55)

#select the 'spec' to animate ...
selected_spec = spec2

#Build the rupture set 
ruptures = [rupt for rupt in sf.get_rupture_ids(selected_spec)]

## Render the 3D visualisation



In [10]:
fig = ipv.figure()
all_tris = np.array(tile_outlines_nztm['triangles'].to_list(), dtype='uint32')
mesh = ipv.plot_trisurf(x=x, y=y, z=z, triangles=all_tris, color='lightblue')
faces = ipv.plot_trisurf(x, y, z, triangles=[[]], color='red')
ipv.ylim( min(y), max(y))
ipv.xlim( min(x), max(x))
ipv.zlim(-250e3, 0)
ipv.show()

In [11]:
# use async co-routines to animate the rupture data 
async def new_faces(n):
    #apply the new filter
    rupt_tris = tile_outlines_nztm[tile_outlines_nztm["FID"].isin(ruptures[n])]
    #convert series to np.array and update the faces
    faces.triangles = np.array(rupt_tris['triangles'].to_list(), dtype='uint32')   

for n in range(len(ruptures)):
    task = asyncio.create_task(new_faces(n))
    await task
    await asyncio.sleep(0.2)
